In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/GTA5.zip'
extract_path = '/content/dataset'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/Cityscapes.zip'
extract_path = '/content/cityscapes'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


Create a dataset whose structure is compatible with what CUT


In [ ]:
import os
import shutil
import random

# --------------------- GTA5 SPLIT ---------------------

# Source path for GTA5 RGB images
gta_path = '/content/dataset/GTA5/images'

# Target paths
trainA_path = 'datasets/gta2cityscapes/trainA'
testA_path  = 'datasets/gta2cityscapes/testA'

# Create output dirs
os.makedirs(trainA_path, exist_ok=True)
os.makedirs(testA_path, exist_ok=True)

# Get and shuffle GTA5 image filenames
gta_images = [f for f in os.listdir(gta_path) if f.endswith('.png')]
random.seed(42)
random.shuffle(gta_images)

# Split into 30/70
split_idx = int(0.3 * len(gta_images))
trainA_imgs = gta_images[:split_idx]
testA_imgs = gta_images[split_idx:]

# Copy files to trainA and testA
for fname in trainA_imgs:
    shutil.copy(os.path.join(gta_path, fname), os.path.join(trainA_path, fname))
for fname in testA_imgs:
    shutil.copy(os.path.join(gta_path, fname), os.path.join(testA_path, fname))

print(f"GTA5: {len(trainA_imgs)} trainA images, {len(testA_imgs)} testA images")

# ------------------ CITYSCAPES SPLIT ------------------

# Source paths for Cityscapes
city_train = '/content/cityscapes/Cityscapes/Cityspaces/images/train'
city_val   = '/content/cityscapes/Cityscapes/Cityspaces/images/val'

# Target paths
trainB_path = 'datasets/gta2cityscapes/trainB'
testB_path  = 'datasets/gta2cityscapes/testB'

os.makedirs(trainB_path, exist_ok=True)
os.makedirs(testB_path, exist_ok=True)

# Copy train images to trainB
for root, _, files in os.walk(city_train):
    for file in files:
        if file.endswith('leftImg8bit.png'):
            src = os.path.join(root, file)
            shutil.copy(src, os.path.join(trainB_path, file))

# Copy val images to testB
for root, _, files in os.walk(city_val):
    for file in files:
        if file.endswith('leftImg8bit.png'):
            src = os.path.join(root, file)
            shutil.copy(src, os.path.join(testB_path, file))

print(f"Cityscapes: trainB has {len(os.listdir(trainB_path))} images")
print(f"Cityscapes: testB has {len(os.listdir(testB_path))} images")


GTA5: 750 trainA images, 1750 testA images
Cityscapes: trainB has 1572 images
Cityscapes: testB has 500 images


Clone and install FASTCUT

In [ ]:
!git clone https://github.com/taesungp/contrastive-unpaired-translation /content/CUT
%cd /content/CUT
!pip install -r requirements.txt

Cloning into '/content/CUT'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 286 (delta 87), reused 82 (delta 82), pack-reused 170 (from 1)
Receiving objects: 100% (286/286), 17.91 MiB | 19.69 MiB/s, done.
Resolving deltas: 100% (150/150), done.
/content/CUT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 30.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:0

training

In [ ]:
!python /content/CUT/train.py \
  --dataroot /content/datasets/gta2cityscapes/ \
  --name gta2cityscapes_fastcut \
  --model cut \
  --CUT_mode fastcut \
  --n_epochs 50 \
  --n_epochs_decay 50 \
  --batch_size 2 \
  --gpu_ids 0 \
  --save_epoch_freq 1 \
  --display_id -1 \
  --load_size 286 \
  --crop_size 143



----------------- Options ---------------
                 CUT_mode: fastcut                       	[default: CUT]
               batch_size: 2                             	[default: 1]
                    beta1: 0.5                           
                    beta2: 0.999                         
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 143                           	[default: 256]
                 dataroot: /content/datasets/gta2cityscapes/	[default: placeholder]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            	[default: None]
            display_ncols: 4                             
             display_port: 8097                          
   

In [ ]:
ls ./checkpoints/gta2cityscapes_cut/

test_opt.txt


In [ ]:
!python /content/CUT/test.py \
  --dataroot /content/datasets/gta2cityscapes/ \
  --name gta2cityscapes_fastcut \
  --CUT_mode fastcut \
  --phase test \
  --no_dropout \
  --num_test 99999 \
  --preprocess resize_and_crop \
  --load_size 286 \
  --crop_size 143 \
  --gpu_ids 0



----------------- Options ---------------
                 CUT_mode: fastcut                       	[default: CUT]
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 143                           	[default: 256]
                 dataroot: /content/datasets/gta2cityscapes/	[default: placeholder]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
               easy_label: experiment_name               
                    epoch: latest                        
                     eval: False                         
        flip_equivariance: True                          
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: xavier                        
                 input_nc: 3    

download the resulting translated dataset

In [ ]:
from google.colab import files

!zip -r /content/test_latest.zip /content/CUT/results/gta2cityscapes_fastcut/test_latest

files.download('/content/test_latest.zip')


Output streaming troncato alle ultime 5000 righe.
  adding: content/CUT/results/gta2cityscapes_fastcut/test_latest/images/real_A/02145.png (deflated 0%)
  adding: content/CUT/results/gta2cityscapes_fastcut/test_latest/images/real_A/02444.png (stored 0%)
  adding: content/CUT/results/gta2cityscapes_fastcut/test_latest/images/real_A/00348.png (deflated 0%)
  adding: content/CUT/results/gta2cityscapes_fastcut/test_latest/images/real_A/01424.png (deflated 0%)
  adding: content/CUT/results/gta2cityscapes_fastcut/test_latest/images/real_A/00652.png (deflated 0%)
  adding: content/CUT/results/gta2cityscapes_fastcut/test_latest/images/real_A/00577.png (deflated 0%)
  adding: content/CUT/results/gta2cityscapes_fastcut/test_latest/images/real_A/00712.png (stored 0%)
  adding: content/CUT/results/gta2cityscapes_fastcut/test_latest/images/real_A/01595.png (deflated 0%)
  adding: content/CUT/results/gta2cityscapes_fastcut/test_latest/images/real_A/00206.png (stored 0%)
  adding: content/CUT/results

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>